In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [ ]:
import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz', spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'}, default_reference='GRCh38') 

In [ ]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

In [ ]:
rpmk = hl.import_bed('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker_all', reference_genome='GRCh38', skip_invalid_intervals=True)

In [ ]:
new_start = hl.if_else(((rpmk.interval.start.position == 1) | (rpmk.interval.start.position == 2)),
                       hl.locus(rpmk.interval.start.contig, rpmk.interval.start.position, reference_genome='GRCh38'),
                       hl.locus(rpmk.interval.start.contig, rpmk.interval.start.position-2, reference_genome='GRCh38'))      

new_end = hl.if_else(((rpmk.interval.end.position == hl.contig_length(rpmk.interval.end.contig, reference_genome='GRCh38')) | (rpmk.interval.end.position+1 == hl.contig_length(rpmk.interval.end.contig, reference_genome='GRCh38'))),
                       hl.locus(rpmk.interval.end.contig, rpmk.interval.end.position, reference_genome='GRCh38'),
                       hl.locus(rpmk.interval.end.contig, rpmk.interval.end.position+2, reference_genome='GRCh38'))
                     
                     

In [ ]:
rpmk = rpmk.annotate(interval_extended = hl.interval(new_start, new_end))

In [ ]:
rpmk.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended.ht')

In [ ]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended.ht')

In [ ]:
rpmk = rpmk.key_by(rpmk.interval_extended)

In [ ]:
rpmk.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended-keyed.ht')

In [ ]:
cov = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/gnomad/gnomad.genomes.r3.0.coverage.summary.tsv.bgz')

In [ ]:
cov = cov.filter(hl.float(cov.over_1) > 0.9)

In [ ]:
cov.checkpoint('/net/scratch/people/plggosborcz/temp-mts/cov-temp.ht')

In [ ]:
cov = hl.read_table('/net/scratch/people/plggosborcz/temp-mts/cov-temp.ht')

In [ ]:
cov = cov.select(cov.locus)

In [ ]:
cov.checkpoint('/net/scratch/people/plggosborcz/temp-mts/cov-loci-90-over-1.ht')

In [ ]:
cov = cov.key_by(cov.locus)

In [ ]:
cov = hl.read_table('/net/scratch/people/plggosborcz/temp-mts/cov-loci-90-over-1.ht')

In [ ]:
cov.show()

In [ ]:
cov = cov.annotate(parsed_locus = hl.parse_locus(cov.locus, reference_genome='GRCh38'))

In [ ]:
cov.write('/net/scratch/people/plggosborcz/temp-mts/gnomad-cov.ht')

In [ ]:
cov = hl.read_table('/net/scratch/people/plggosborcz/temp-mts/gnomad-cov.ht')

In [ ]:
cov = cov.key_by(cov.parsed_locus)

In [ ]:
cov.write('/net/scratch/people/plggosborcz/temp-mts/gnomad-cov-keyed.ht')